# Step49, Dataset 클래스와 전처리 

이전 단계에서 스파이럴 데이터셋을 읽어 올때 x,t = dezero.datasets.get_spiral() 사용  
이때 x와 t는 ndarray 인스턴스  
즉, 총 300개의 데이터를 하나의 ndarray 인스턴스에 담았다.

하지만 대규모 데이터셋을 처리하는 경우  
거대한 데이터를 하나의 ndarray 인스턴스로 처리하면 모든 원소를 한꺼번에 메모리에 올려야한다.

이러한 문제에 대응할 수 있도록 데이터셋 전용 클래스인 Dataset 클래스를 만든다.

## 49.1 Dataset 클래스 구현 

Dataset 클래스는 기반 클래스로서의 역할,  
사용자가 실제로 사용하는 데이터셋은 이를 상속하여 구현

**진짜 중요**

In [1]:
# dezero/datasets.py
import numpy as np 

class Dataset:
    def __init__(self, train=True):
        self.train = train 
        self.data = None 
        self.label = None 
        self.prepare() 

    def __getitem__(self, index):
        assert np.isscalar(index)   # index는 정수(스칼라)만 지원   , assert는 step5to7확인
        if self.label is None:
            return self.data[index], None
        else:
            return self.data[index], self.label[index]
    
    def __len__(self):
        return len(self.data)
    
    def prepare(self):
        pass

초가화 때 train 인수로 받음, '학습'이냐 '테스트'냐를 구별하기위한 플래그  
인스턴스 변수 data, label : 각각 입력 데이터와 레이블을 보관  
prepare 메서드 : 자식 클래스에서는 prepare 메서드가 데이터 준비 작업을 하도록 구현  

Dataset 클래스에서 가장 중요한 메서드 \__getitem\__ 과 \__len\__  
이 두 메서드를 제공해야만 '데이터셋'

\__getitem\__ : 파이썬의 특수 메서드  
x\[0\], x\[1\]처럼 괄호를 사용해 접근할 때의 동작을 정의  
Dataset 클래스의 \__getitem\__ 메서드 : 단순히 지정된 인덱스에 위치하는 데이터를 꺼낸다.  
레이블 데이터가 없는 경우 : 입력 데이터 self.data\[index\]의 레이블은 None을 반환한다.

\__len\__ 메서드 : 데이터셋의 길이를 알려줌, len() 함수를 사용할때 호출 (예: len(x))

Dataset 클래스를 상속하여 스파이럴 데이터셋을 구현해본다.

In [2]:
# dezero/datasets.py 
class Spiral(Dataset):
    def prepare(self):
        self.data, self.label = get_spiral(self.train)

In [3]:
import dezero 

train_set = dezero.datasets.Spiral(train=True)
print(train_set[0]) # 0번째 입력 데이터와 레이블이 튜플로 반환
print(len(train_set))

(array([-0.13981389, -0.00721657], dtype=float32), 1)
300


## 49.2 큰 데이터셋의 경우

스파이럴 데이터셋 같은 작은 데이터셋이라면 Dataset 클래스의 인스턴스 변수인 data와 label에 직접 ndarray 인스턴스를 유지해도 무리가 없다.

하지만 데이터셋이 훨씬 크면 이런 구현 방식은 사용할수 없다.

In [4]:
class BigData(Dataset):
    def __getitem__(index):
        x = np.load('data/{}.npy'.format(index))
        t = np.load('label/{}.npy'.format(index))
        return x,t
    
    def __len__():
        return 1000000

data 디렉터리와 label 디렉터리에 각각 100만 개의 데이터가 저장되어 있다고 가정 

BigData 클래스를 초기화 할때는 데이터를 읽지 않고  
데이터에 접근할 때 비로소 읽게 한다. 구체적으로 \__getitem()\__(index)가 불리는 시점에서 data 디렉터리에 있는 데이터를 읽는다.



이어서 Spiral 클래스를 사용하여 학습 코드를 작성해본다. 그러려면 데이터를 '연결'해야한다.


## 49.3 데이터 이어 붙이기 

신경망을 학습시킬 때는 데이터셋 중 일부를 미니배치로 꺼낸다.

다음은 Spiral 클래스를 사용하여 데이터를 미니배치로 가져오는 코드이다.

In [5]:
train_set = dezero.datasets.Spiral()

batch_index = [0,1,2]   # 0에서 2번째까지의 데이터만 꺼냄
batch = [train_set[i] for i in batch_index]

x = np.array([example[0] for example in batch])
t = np.array([example[1] for example in batch])

print(x.shape)
print(t.shape)

(3, 2)
(3,)


인덱스를 지정하여 여러 데이터(미니배치)를 꺼낸다.  
batch에 여러 데이터가 리스트로 저장된다

이제 이 데이터를 하나의 ndarray 인스턴스로 변환한다.  
batch의 각 원소에서 데이터(혹은 레이블)만을 꺼내 하나의 ndarray 인스턴스로 변형했다(이어 붙였다)

이것으로 신경망에 입력할 수 있는 형태의 데이터가 마련되었다.

## 49.4 학습 코드 


In [6]:
import numpy as np 
import dezero
import math
from dezero import optimizers
from dezero.models import MLP
import dezero.functions as F

# 하이퍼 파라메터 설정
max_epoch = 300 
batch_size = 30 
hidden_size = 10 
lr = 1.0 

# 데이터 읽기, 모델, 옵티마이저 생성
train_set = dezero.datasets.Spiral()
model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)

data_size = len(train_set)
max_iter = math.ceil(data_size / batch_size)

for epoch in range(max_epoch):
    index = np.random.permutation(data_size)
    sum_loss = 0

    for i in range(max_iter):
        # 미니배치 꺼내기 
        batch_index = index[i * batch_size : (i+1) * batch_size]
        batch = [train_set[i] for i in batch_index]
        batch_x = np.array([example[0] for example in batch])
        batch_t = np.array([example[1] for example in batch])

        y = model(batch_x)
        loss = F.softmax_cross_entropy(y, batch_t)
        model.cleargrads()
        loss.backward()
        optimizer.update()

        sum_loss += float(loss.data) * len(batch_t)
    
    # 에포크마다 손실 출력 
    avg_loss = sum_loss / data_size 
    print('epoch %d, loss %.2f' % (epoch + 1, avg_loss))

epoch 1, loss 1.13
epoch 2, loss 1.05
epoch 3, loss 0.95
epoch 4, loss 0.92
epoch 5, loss 0.87
epoch 6, loss 0.89
epoch 7, loss 0.84
epoch 8, loss 0.78
epoch 9, loss 0.80
epoch 10, loss 0.79
epoch 11, loss 0.78
epoch 12, loss 0.76
epoch 13, loss 0.77
epoch 14, loss 0.76
epoch 15, loss 0.76
epoch 16, loss 0.77
epoch 17, loss 0.78
epoch 18, loss 0.74
epoch 19, loss 0.74
epoch 20, loss 0.72
epoch 21, loss 0.73
epoch 22, loss 0.74
epoch 23, loss 0.77
epoch 24, loss 0.73
epoch 25, loss 0.74
epoch 26, loss 0.74
epoch 27, loss 0.72
epoch 28, loss 0.72
epoch 29, loss 0.72
epoch 30, loss 0.73
epoch 31, loss 0.71
epoch 32, loss 0.72
epoch 33, loss 0.72
epoch 34, loss 0.71
epoch 35, loss 0.72
epoch 36, loss 0.71
epoch 37, loss 0.71
epoch 38, loss 0.70
epoch 39, loss 0.71
epoch 40, loss 0.70
epoch 41, loss 0.71
epoch 42, loss 0.70
epoch 43, loss 0.70
epoch 44, loss 0.70
epoch 45, loss 0.69
epoch 46, loss 0.69
epoch 47, loss 0.71
epoch 48, loss 0.70
epoch 49, loss 0.69
epoch 50, loss 0.69
epoch 51,

이제 Dataset 클래스를 사용하여 신경망을 학습할 수 있게 되었다.  
Dataset 클래스를 사용하는 이점은 다른 데이터셋으로 교체해 학습할 때 확인할 수 있다.  
데이터셋 인터페이스를 통일하여 다양한 데이터셋을 똑같은 코드로 처리할 수 있다.

## 49.5 데이터셋 전처리 

머신러닝에서는 모델에 데이터를 입력하기 전에 데이터를 특정한 형태로 가공할 일이 많다.  

데이터에서 특정 값을 제거  
데이터의 형상을 변형하는 처리  
이미지를 회전 혹은 좌우 바전, 데이터 수를 인위적으로 늘리는 기술 : data augmentation

이런 전처리(및 데이터 확장)에 대응하기 위해 Dataset 클래스에도 전처리 기능을 추가한다.

In [7]:
# dezero/datasets.py 

class Dataset:
    def __init__(self, train=True, transform=None, target_transform=None):
        self.train = train 
        self.transform = transform
        self.target_transform = target_transform

        if self.transform is None:
            self.transform = lambda x: x    # 받은 인수를 그대로 반환
        if self.target_transform is None:
            self.target_transform = lambda x: x  # 받은 인수를 그대로 반환
        
        self.data = None 
        self.label = None 
        self.prepare()

    def __getitem__(self, index):
        assert np.isscalar(index)
        if self.label is None:
            return self.transform(self.data[index]), None 
        else:
            return self.transform(self.data[index]), self.target_transform(self.label[index])
    
    def __len__(self):
        return len(self.data)
    
    def prepare(self):
        pass

transform, target_transform 인수들은 호출 가능한 객체를 받는다.

transform : 입력 데이터 하나에 대한 변환을 처리  
target_transform : 레이블 하나에 대한 변환을 처리 

In [8]:
# 입력 데이터를 1/2로 스케일 변환하는 전처리 
def f(x):
    y = x / 2.0
    return y 

train_set = dezero.datasets.Spiral(transform=f)

데이터 정규화(normalizarion)와  
이미지 데이터(PIL.Image 인스턴스) 변환 등 전처리 시 사용되는 변환들을  
dezero / transforms.py에 있다.

In [9]:
# 입력을 x라 할때 (x - mean) / std 변환
from dezero import transforms

f = transforms.Normalize(mean=0.0, std=2.0)
train_set = dezero.datasets.Spiral(transform=f)

In [10]:
# 변환 처리를 연달아 수행하기 
from dezero import transforms

f = transforms.Compose([transforms.Normalize(mean=0.0, std=2.0), transforms.AsType(np.float64)])
# Compose 클래스는 주어진 변환 목록을 앞에서부터 순서대로 처리
# 정규화(Normalize) -> 데이터 타입 np.float64변환(AsType)